# Nueral architecture search on minst dataset

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak


2023-03-21 21:44:23.180790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 21:44:24.013225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 21:44:24.013286: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 21:44:25.820364: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Loading data
Here load the mnist data into training and testing samples

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


# Training
 Here train the autokeras built in image clasfier with mnist data after the training the validation accuracy of the architecture is 98 %

In [10]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 06m 51s]
val_loss: 0.042387738823890686

Best val_loss So Far: 0.042387738823890686
Total elapsed time: 00h 06m 51s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.1546 - accuracy: 0.9532
Epoch 2/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0721 - accuracy: 0.9779
Epoch 3/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0573 - accuracy: 0.9822
Epoch 4/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0505 - accuracy: 0.9842
Epoch 5/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0436 - accuracy: 0.9863
Epoch 6/10
1875/1875 [==============================] - 50s 26ms/step - loss: 0.0403 - accuracy: 0.9871
Epoch 7/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0353 - accuracy: 0.9894
Epoch 8/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0349 - accuracy: 0.9888
Epoch 9/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0317 - accuracy: 0.9891
Epoch 10/10
1875/1875 [==============================] - 50s 27m

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


313/313 [==============================] - 2s 5ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
313/313 [==============================] - 2s 5ms/step - loss: 0.0362 - accuracy: 0.9887
[0.03616759181022644, 0.9886999726295471]


# Customized Search Space

here only resnet archirtures will be searched for classification on mnist data will be train.

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [4]:
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [04h 43m 01s]
val_loss: 0.06486619263887405

Best val_loss So Far: 0.06486619263887405
Total elapsed time: 04h 43m 01s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 2046s 1s/step - loss: 0.3450 - accuracy: 0.9168
Epoch 2/10
1875/1875 [==============================] - 2036s 1s/step - loss: 0.1741 - accuracy: 0.9604
Epoch 3/10
1875/1875 [==============================] - 2039s 1s/step - loss: 0.2676 - accuracy: 0.9439
Epoch 4/10
1875/1875 [==============================] - 2040s 1s/step - loss: 0.1767 - accuracy: 0.9560
Epoch 5/10
1875/1875 [==============================] - 2045s 1s/step - loss: 0.2132 - accuracy: 0.9497
Epoch 6/10
1875/1875 [==============================] - 2043s 1s/step - loss: 0.1179 - accuracy: 0.9700
Epoch 7/10
1875/1875 [==============================] - 2046s 1s/step - loss: 0.1320 - accuracy: 0.9692
Epoch 8/10
1875/1875 [==============================] - 2049s 1s/step - loss: 0.0977 - accuracy: 0

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


Here by using AutoKeras to create an Image Classification model, but with a slightly different custom search space like The ResNetBlock layer is added to the output node. In this case, the version argument is set to "v2", which specifies the version of the ResNet architecture to use. An AutoModel is created using the input and output nodes.

In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

The overwrite argument is set to True, which means that if there is an existing model with the same name, it will be overwritten. The max_trials argument is set to 1, which means that AutoKeras will run 1 trials to find the best model architecture.

In [5]:
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [07h 48m 30s]
val_loss: 4.578865051269531

Best val_loss So Far: 4.578865051269531
Total elapsed time: 07h 48m 30s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 3580s 2s/step - loss: 0.2054 - accuracy: 0.9427
Epoch 2/10
1875/1875 [==============================] - 3548s 2s/step - loss: 0.1375 - accuracy: 0.9671
Epoch 3/10
1875/1875 [==============================] - 4527s 2s/step - loss: 0.1028 - accuracy: 0.9740
Epoch 4/10
1875/1875 [==============================] - 3477s 2s/step - loss: 0.0775 - accuracy: 0.9800
Epoch 5/10
1875/1875 [==============================] - 3392s 2s/step - loss: 0.0682 - accuracy: 0.9813
Epoch 6/10
1875/1875 [==============================] - 3394s 2s/step - loss: 0.0642 - accuracy: 0.9837
Epoch 7/10
1875/1875 [==============================] - 3395s 2s/step - loss: 0.0425 - accuracy: 0.9880
Epoch 8/10
1875/1875 [==============================] - 5160s 3s/step - loss: 0.0364 - accuracy: 0.9894
Epoch 9/10
1875/1875 [==============================] - 5909s 3s/step - loss: 0.0310 - accuracy: 0.9911
Epoch 10/10
1875/1875 [==============================] - 5899s 3

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


# AutoKeras image classifier for STL10 
For train the image classifier on STL10 dataset 

Import the necessary libraries: numpy, torch, torchvision.datasets, torchvision.transforms, and autokeras.ImageClassifier.

In [1]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from autokeras import ImageClassifier


2023-03-30 07:31:15.838983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 07:31:18.406255: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-30 07:31:18.406376: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-30 07:31:18.406383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Check if a GPU is available and set the device to GPU if it is available; otherwise, set the device to CPU.

In [2]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


Define the transformation to be applied to the images in the dataset. In this case, the images will be converted to tensors and normalized with a mean of 0.5 and a standard deviation of 0.5

In [3]:
# Define the transform for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


Load the STL10 dataset using the DataLoader class from torchvision.datasets. The dataset is split into a training set and a test set. The train_loader and test_loader are created using the DataLoader class with the dataset, batch size of 6, and shuffle set to True for the training set and False for the test set.

In [4]:
# Load the STL10 dataset using the DataLoader class
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=6, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Extract the data and labels from the train_loader and test_loader and concatenate them into x_train, y_train, x_test, and y_test arrays

In [8]:
# Extract the data and labels from the loaders
x_train = []
y_train = []
for data, label in train_loader:
    x_train.append(data.numpy())
    y_train.append(label.numpy())
x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

x_test = []
y_test = []
for data, label in test_loader:
    x_test.append(data.numpy())
    y_test.append(label.numpy())
x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# Move the data to GPU if available
#x_train = torch.from_numpy(x_train).to(device)
#y_train = torch.from_numpy(y_train).to(device)
#x_test = torch.from_numpy(x_test).to(device)
#y_test = torch.from_numpy(y_test).to(device)

Define the ImageClassifier model from AutoKeras with a max_trials value of 1, which means that AutoKeras will only try one set of hyperparameters.

In [ ]:
# Define the model
clf = ImageClassifier(max_trials=1, overwrite=True, seed=42)


Train the model using the fit method with the x_train and y_train arrays and 10 epochs. Then evaluate the model on the test data using the evaluate method with the x_test and y_test arrays.
Finally print the test accuracy of the model.

In [1]:
# Train the model
clf.fit(x_train, y_train, epochs=10)
# Evaluate the model
score = clf.evaluate(x_test, y_test)
print('Test accuracy:', score[1])

Trial 1 Complete [00h 01m 26s]
val_loss: 1.6100943088531494

Best val_loss So Far: 1.6100943088531494
Total elapsed time: 00h 01m 26s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
157/157 [==============================] - 9s 55ms/step - loss: 1.9860 - accuracy: 0.2772
Epoch 2/10
157/157 [==============================] - 9s 54ms/step - loss: 1.7174 - accuracy: 0.3796
Epoch 3/10
157/157 [==============================] - 8s 51ms/step - loss: 1.6180 - accuracy: 0.4186
Epoch 4/10
157/157 [==============================] - 9s 55ms/step - loss: 1.5042 - accuracy: 0.4526
Epoch 5/10
157/157 [==============================] - 8s 52ms/step - loss: 1.4325 - accuracy: 0.4816
Epoch 6/10
157/157 [==============================] - 8s 54ms/step - loss: 1.3742 - accuracy: 0.5010
Epoch 7/10
157/157 [==============================] - 8s 52ms/step - loss: 1.3211 - accuracy: 0.5178
Epoch 8/10
157/157 [==============================] - 8s 53ms/step - loss: 1.2500 - accuracy: 0.5562
Epoch 9/10
157/157 [

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


250/250 [==============================] - 5s 13ms/step - loss: 1.5903 - accuracy: 0.4604
Test accuracy: 0.46037501096725464


In [8]:
# Evaluate the AutoModel on the test data
accuracy = clf.evaluate(x_test, y_test)[1]
print('Accuracy: {:.2f}%'.format(accuracy * 100))

250/250 [==============================] - 5s 13ms/step - loss: 1.5903 - accuracy: 0.4604
Accuracy: 46.04%


Here Export Model and print the summary of model. The output shows that the exported model is a Keras functional model, which is a type of model that allows for more complex architectures than the simple sequential model.

The model has several layers, including convolutional layers, pooling layers, and a dense layer, with a total of 107,819 parameters. The model is trained to classify the STL-10 dataset into 10 classes.

In [10]:
# Export as a Keras Model.
model = clf.export_model()
#print(model)
print(type(model))  
model.summary()

<class 'keras.engine.functional.Functional'>
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 96, 96)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 3, 96, 96)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 3, 96, 96)        193       
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 3, 96, 32)         27680     
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 96, 64)         18496     
                                                                 
 max_pooling2d (

# AutoModel

Define the AutoModel using the ImageInput and ImageBlock classes from AutoKeras to define the input and convolutional layers of the neural network respectively. Then add a Flatten layer to convert the output of the convolutional layers into a 1D tensor, and a DenseBlock to add some fully connected layers to the network. Finally, add a ClassificationHead layer to the network to output the classification probabilities.

In [9]:
import autokeras as ak
# Define the AutoModel
input_node = ak.ImageInput()
output_node = ak.ImageBlock()(input_node)
output_node = ak.Flatten()(output_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

Initialize the AutoModel by passing in the input and output nodes defined in the previous step, along with some additional parameters. Set overwrite to True to allow the model to overwrite existing files, and set max_trials to 2 to limit the number of different architectures that the model will try.

In [10]:

# Initialize the AutoModel
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=2
)

Train the AutoModel using the fit method, passing in the training data x_train and y_train, and the number of epochs to train for.Evaluate the AutoModel on the test data using the evaluate method, passing in x_test and y_test. Save the resulting accuracy score in a variable named accuracy.

In [11]:
clf.fit(x_train, y_train, epochs=10)
# Evaluate the AutoModel on the test data
accuracy = clf.evaluate(x_test, y_test)[1]
print('Accuracy: {:.2f}%'.format(accuracy * 100))

Trial 2 Complete [00h 33m 27s]
val_loss: 2.3032314777374268

Best val_loss So Far: 2.3030271530151367
Total elapsed time: 01h 02m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
157/157 [==============================] - 379s 2s/step - loss: 2.3724 - accuracy: 0.0924
Epoch 2/10
157/157 [==============================] - 362s 2s/step - loss: 2.3029 - accuracy: 0.0970
Epoch 3/10
157/157 [==============================] - 364s 2s/step - loss: 2.3028 - accuracy: 0.0996
Epoch 4/10
157/157 [==============================] - 364s 2s/step - loss: 2.3029 - accuracy: 0.0932
Epoch 5/10
157/157 [==============================] - 362s 2s/step - loss: 2.3029 - accuracy: 0.0914
Epoch 6/10
157/157 [==============================] - 360s 2s/step - loss: 2.3028 - accuracy: 0.0984
Epoch 7/10
157/157 [==============================] - 363s 2s/step - loss: 2.3029 - accuracy: 0.0950
Epoch 8/10
157/157 [==============================] - 363s 2s/step - loss: 2.3029 - accuracy: 0.0978
Epoch 9/10
157/157 [

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
2023-03-30 09:38:00.994674: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 884736000 exceeds 10% of free system memory.
2023-03-30 09:38:12.569548: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 884736000 exceeds 10% of free system memory.
2023-03-30 09:38:13.241355: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 884736000 exceeds 10% of free system memory.


250/250 [==============================] - 53s 198ms/step - loss: 2.3026 - accuracy: 0.1000
Accuracy: 10.00%


Here is the summary of model 

In [12]:
# Export as a Keras Model.
model = clf.export_model()
#print(model)
print(type(model))  
model.summary()

<class 'keras.engine.functional.Functional'>
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 96, 96)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 3, 96, 96)        0         
 t32)                                                            
                                                                 
 resizing (Resizing)         (None, 32, 96, 96)        0         
                                                                 
 conv2d (Conv2D)             (None, 32, 96, 3)         291       
                                                                 
 resnet50 (Functional)       (None, 1, 3, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                

# For DTD Dataset

 # AutoModel
Here the task is Image classification using AutoKeras, Specifically an image classifier using a ResNet architecture on DTD and utilizes AutoKeras to automatically search for the best hyperparameters and architecture

Import necessary libraries including numpy, torch, datasets and transforms from torchvision, and the ImageClassifier from autokeras.

In [1]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from autokeras import ImageClassifier

2023-03-30 17:21:25.343334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 17:21:28.136195: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-30 17:21:28.136272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-30 17:21:28.136279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Check if a GPU is available and if not, fall back to using the CPU.

In [2]:
# Check if a GPU is available, if not fall back to CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define the transform for the dataset, including resizing the images to 224x224 pixels, converting them to tensors, and normalizing them.

In [3]:
# Define the transform for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # add resize operation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Load the train and test datasets using the ImageFolder class from PyTorch's torchvision module and create data loaders using the DataLoader class.

In [4]:
# Load the DTD dataset using the DataLoader class
train_dataset = datasets.ImageFolder(root='/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/classification/DTD/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/classification/DTD/val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: torch.utils.data.dataloader.default_collate([y for y in x]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: torch.utils.data.dataloader.default_collate([y for y in x]))


Concatenate the data and labels from the loaders into numpy arrays for use with AutoKeras.

In [5]:
# Extract the data and labels from the loaders
x_train = np.concatenate([data.numpy() for data, label in train_loader], axis=0)
y_train = np.concatenate([label.numpy() for data, label in train_loader], axis=0)

x_test = np.concatenate([data.numpy() for data, label in test_loader], axis=0)
y_test = np.concatenate([label.numpy() for data, label in test_loader], axis=0)

Define the AutoKeras model using an ImageInput node, an ImageBlock node with a ResNet architecture and no data augmentation, and a ClassificationHead node.Use the AutoModel class to create the model and fit it to the training data for 10 epochs with a maximum of 3 trials.

In [6]:
import autokeras as ak
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [ ]:
clf.fit(x_train, y_train, epochs=5)

Trial 1 Complete [00h 59m 49s]
val_loss: 3.8677918910980225

Best val_loss So Far: 3.8677918910980225
Total elapsed time: 00h 59m 49s
INFO:tensorflow:Oracle triggered exit


Export the model as a Keras model and print its summary.

In [ ]:
# Export as a Keras Model.
model = clf.export_model()
#print(model)
print(type(model))  
model.summary()